In [ ]:
from SOView import *
import shutil
import os
import gc
from scipy import sparse

In [ ]:
dataset_start_id = 132

In [ ]:
dataset_list = ['Sun2021Integrating']

In [ ]:
dataset_species_list = [
    'Mouse',



 
    
]

In [ ]:
dataset_tissue_list = [
    'visual cortex',


]

In [ ]:
len(dataset_list)

In [ ]:
ifsparse =False
# is the expression matrix sparse?
raw_count = True
# is the expression matrix presented as raw count?

In [ ]:


def SODB_process(root_path,source_h5ad_path,biotech,biotech_category,dataset_short):
    # 多个单词要加下划线

    h5ad_list = os.listdir(source_h5ad_path)
    out_dataname_list = []
    out_h5ad_path_list = []
    out_feature_dict_list = []
    N_Unit_list = []
    N_Gene_list = []
    for hl in h5ad_list:
        if not hl.endswith('.h5ad'):
            continue
        # source_h5ad_data = hl.split('.')[0]
        source_h5ad_data = hl[0:-5]


        source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'
        print(source_h5ad)
        data_name = source_h5ad_data


        path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
        path_abs = root_path+path_rel
        os.makedirs(path_abs, exist_ok=False)
        print("The new directory is created!")
        shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')


        data_use = path_rel
        adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
        adata.var_names_make_unique()


        # print(adata.X)
        print(f'h5ad path: {data_use}/data.h5ad')
        if 'in_tissue' in adata.obs:
            adata = adata[adata.obs['in_tissue']==1]
        adata.obsm['spatial'] = np.array(adata.obsm['spatial'])

        
        
        adata.layers["counts"] = adata.X.copy()

        # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
        if raw_count:
            # is all the element in adata.X are integer?

            sc.pp.normalize_total(adata, inplace=True)
            sc.pp.log1p(adata)
        if adata.shape[1]>=2000:
            sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
        else:
            pass


        sc.pp.pca(adata)
        sc.pp.neighbors(adata)
        sc.tl.leiden(adata)

        sc.pl.embedding(adata,basis='spatial',color='leiden')
        sc.tl.umap(adata,n_components=3)
        SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
        print(f'View path: {data_use}/View_UMAP_LAB.pdf')

        gene_list = np.array(adata.var_names).tolist()
        obs_list = np.array(adata.obs.keys()).tolist()
        obsm_list = []
        for obsm in np.array(adata.obsm.keys()).tolist():
            for i in range(adata.obsm[obsm].shape[1]):
                cur_obsm = f'{obsm}@{i}'
                obsm_list.append(cur_obsm)

        feature_dict = {
            'gene':gene_list,
            'obs':obs_list,
            'obsm':obsm_list
        }

        import pickle
        with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
            pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
        sq.gr.spatial_neighbors(adata, coord_type="generic",n_neighs=10)
        sq.gr.spatial_autocorr(adata, mode="moran")
        
        if ifsparse:
            adata.X = sparse.csr_matrix(adata.layers["counts"])
        lay_keys = list(adata.layers.keys())
        for lay in lay_keys:
            del adata.layers[lay]
        
        adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
        out_dataname_list.append(source_h5ad_data)
        out_h5ad_path_list.append(f'{data_use}/data.h5ad')
        out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
        N_Unit_list.append(adata.shape[0])
        N_Gene_list.append(adata.shape[1])

        print(source_h5ad_data)
        print(f'h5ad path: {data_use}/data.h5ad')
        print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
        print(f'N_Unit: {adata.shape[0]}')
        print(f'N_Gene: {adata.shape[1]}')
        del adata
        gc.collect()
    return out_dataname_list,out_h5ad_path_list,out_feature_dict_list,N_Unit_list,N_Gene_list

In [ ]:
out_dataname_list_list = []
out_h5ad_path_list_list = []
out_feature_dict_list_list = []
N_Unit_list_list = []
N_Gene_list_list = []
tissue_list = []
species_list = []
dataset_id_list = []
biotech_id_list = []

In [ ]:
root_path = 'BARISTASeq0928/'
source_h5ad_root = 'BARISTASeq'
# dataset_short = 'asp2017spatial'

biotech = 'BARISTASeq'
biotech_category = 'Spatial_Transcriptomics'



for i in range(len(dataset_list)):
    dataset_short = dataset_list[i]

    cur_datasetid = i+dataset_start_id
    source_h5ad_path = f'{source_h5ad_root}'
    biotech_id = 31



    out_dataname_list,out_h5ad_path_list,out_feature_dict_list,N_Unit_list,N_Gene_list = SODB_process(root_path,source_h5ad_path,biotech,biotech_category,dataset_short)
    tissue = [dataset_tissue_list[i]]*len(out_dataname_list)
    species = [dataset_species_list[i]]*len(out_dataname_list)
    # dataset_id_list = [cur_datasetid]
    
    out_dataname_list_list.append(out_dataname_list)
    out_h5ad_path_list_list.append(out_h5ad_path_list)
    out_feature_dict_list_list.append(out_feature_dict_list)
    N_Unit_list_list.append(N_Unit_list)
    N_Gene_list_list.append(N_Gene_list)
    tissue_list.append(tissue)
    species_list.append(species)

    dataset_id_list.append([cur_datasetid]*len(out_dataname_list))
    biotech_id_list.append([biotech_id]*len(out_dataname_list))

In [ ]:
1+1

In [ ]:
def print_list_list(ll):
    for l in ll:
        for i in l:
            print(i)

In [ ]:
print_list_list(out_dataname_list_list)

In [ ]:
print_list_list(biotech_id_list)

In [ ]:
print_list_list(dataset_id_list)

In [ ]:
print_list_list(out_h5ad_path_list_list)

In [ ]:
print_list_list(out_feature_dict_list_list)

In [ ]:
print_list_list(species_list)

In [ ]:
print_list_list(tissue_list)

In [ ]:
print_list_list(N_Unit_list_list)

In [ ]:
print_list_list(N_Gene_list_list)